Diabetes Dataset

Descrição: Dados de saúde para prever se pacientes têm diabetes com base em indicadores como pressão sanguínea, idade e nível de glicose. A coluna Outcome é a target: 1 é para positivo para diabetes, 0 para negativo.

Sugestões de Análise:
Exploração de dados: Analise as distribuições e correlações entre variáveis.

Classificação:
Use Regressão Logística e Random Forest para previsões. Teste SVM e KNN para comparação.

Avaliação: Utilize métricas como Acurácia, Recall e F1-score.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score

In [16]:
# Importing dataset and taking a first look at it
dataset = pd.read_csv(r'diabetes.csv')

dataset.info()

print('')
print('Checking for any missing values in the dataset:')
dataset.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB

Checking for any missing values in the dataset:


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64